## Completar Página 12

En esta notebook arreglamos los datos de Página12 que están mal bajados

In [1]:
import pandas as pd

df = pd.read_excel("../data/pagina_12.xlsx", header=0)

print(f"Tenemos {df.shape[0]} noticias")

Tenemos 810 noticias


In [35]:
df["titulo_arreglado"] = df["titulo"]

In [2]:
df.iloc[0]

medio                                             pagina 12
url       https://elplanetaurbano.com/2019/09/top-5-dest...
fecha                                   2019-09-21 10:00:29
titulo                            TOP 5 DESTINOS CANNÁBICOS
Name: 0, dtype: object

Busquemos los índices incompletos

In [39]:
from tqdm.auto import tqdm

incomplete_idx = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    title = row["titulo"]
    if "..." in title:
        incomplete_idx.append(idx)

In [40]:
len(incomplete_idx)

368

Bajemos las noticias


In [5]:
from newspaper import Article

articles = {}

for idx in tqdm(incomplete_idx):
    row = df.loc[idx]
    article = Article(row.url)
    article.download()
    article.parse()
    
    articles[idx] = article

In [6]:
art = articles[3]

In [13]:
type(art.clean_doc)

lxml.html.HtmlElement

In [15]:
doc = art.doc

doc

<Element html at 0x7efc6589a4f0>

Lo meto en otra columna

In [36]:
for idx in articles:
    art = articles[idx]
    title_meta = art.doc.cssselect('meta[property="og:title"]')[0]
    fixed_title = title_meta.get('content')
    
    df.loc[idx, "titulo_arreglado"] = fixed_title

Lo salvamos!

In [42]:
df.to_csv("../data/pagina_12_titulos_arreglados.csv")